In [21]:
from numpy.random import seed
seed(1)

In [22]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [23]:
import numpy as np
import pandas as pd

In [27]:
x_df = pd.read_csv("../reduced_data/steam_all_param_merged_nan_removed_newclasses.csv")
x_df.head()

,appid,metascore,recommendations,positive_x,negative_x,developer,publisher,maxccu,success_class,suc_class,...,tagphilisophical,taggaming,tagfaith,tagmysterydungeon,tagfeaturefilm,tagsteammachine,tagclicker,taghardware,tag360video,tagvronly
0,10,88,100006.0,169280.0,4360.0,Valve,Valve,31930,A,1,...,0,0,0,0,0,0,0,0,0,0
1,20,0,3754.0,4628.0,800.0,Valve,Valve,236,F,0,...,0,0,0,0,0,0,0,0,0,0
2,30,79,2779.0,4527.0,493.0,Valve,Valve,364,F,0,...,0,0,0,0,0,0,0,0,0,0
3,40,0,1276.0,1651.0,353.0,Valve,Valve,629,F,0,...,0,0,0,0,0,0,0,0,0,0
4,50,0,8162.0,9938.0,520.0,Gearbox Software,Valve,620,F,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
x_df["developer"]=x_df["developer"].astype("category")
x_df["developer"]=x_df["developer"].cat.codes
x_df["publisher"]=x_df["publisher"].astype("category")
x_df["publisher"]=x_df["publisher"].cat.codes
x_df.head()

,appid,metascore,recommendations,positive_x,negative_x,developer,publisher,maxccu,success_class,suc_class,...,tagphilisophical,taggaming,tagfaith,tagmysterydungeon,tagfeaturefilm,tagsteammachine,tagclicker,taghardware,tag360video,tagvronly
0,10,88,100006.0,169280.0,4360.0,18255,15348,31930,A,1,...,0,0,0,0,0,0,0,0,0,0
1,20,0,3754.0,4628.0,800.0,18255,15348,236,F,0,...,0,0,0,0,0,0,0,0,0,0
2,30,79,2779.0,4527.0,493.0,18255,15348,364,F,0,...,0,0,0,0,0,0,0,0,0,0
3,40,0,1276.0,1651.0,353.0,18255,15348,629,F,0,...,0,0,0,0,0,0,0,0,0,0
4,50,0,8162.0,9938.0,520.0,6704,15348,620,F,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
reduced_x = x_df.drop(columns=["positive_x", "negative_x", "appid", "metascore", "recommendations", "maxccu", "success_class",
                              "suc_class", "followers", "positive_y", "negative_y"])
reduced_x

,developer,publisher,maxprice,maxdiscount,totaltags,tagstrategy,tagaction,tagadventure,tagdesignillustration,tagutilities,...,tagphilisophical,taggaming,tagfaith,tagmysterydungeon,tagfeaturefilm,tagsteammachine,tagclicker,taghardware,tag360video,tagvronly
0,18255,15348,999,90,31989,602,5337,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18255,15348,499,90,3123,0,738,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18255,15348,499,90,4836,14,156,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,18255,15348,499,90,1318,0,626,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6704,15348,499,90,3163,0,312,109,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31681,7882,6697,99,40,2374,0,0,0,0,0,...,0,0,0,0,0,0,183,0,0,0
31682,3589,5609,99,0,537,20,72,65,0,0,...,0,0,0,0,0,0,0,0,0,0
31683,19968,16855,999,0,2363,0,214,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31684,349,353,0,0,8373,0,0,476,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
X = reduced_x
y = x_df["success_class"]
y.head()

0    A
1    F
2    F
3    F
4    F
Name: success_class, dtype: object

In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [40]:
X_train.shape, y_train.shape

((23764, 346), (23764,))

In [41]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [42]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [43]:
for label, original_class in zip(encoded_y, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: A
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 3
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: C
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: C
Encoded Label: 2
------------
Original Class: B
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: D
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: A
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
-----

Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: D
Encoded Label: 4
------------
Original Class: F
Encoded Label: 3
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original C

------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: D
Encoded Label: 4
------------
Original Class: C
Encoded Label: 4
------------
Original Class: B
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: B
Encoded Label: 4
------------
Original Class: D
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded L

Encoded Label: 4
------------
Original Class: C
Encoded Label: 4
------------
Original Class: F
Encoded Label: 3
------------
Original Class: C
Encoded Label: 4
------------
Original Class: A
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: D
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: A
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: C
Encoded Label: 0
------------
Original C

------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 3
------------
Original Class: F
Encoded Label: 4
------------
Original Class: C
Encoded Label: 4
------------
Original Class: D
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 0
------------
Original Class: D
Encoded Label: 4
------------
Original Class: D
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded L

...

Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 3
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 0
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
-----

Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 3
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original C

------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 3
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 3
------------
Original Class: F
Encoded Label: 3
------------
Original Class: F
Encoded L

------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded L

------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded Label: 4
------------
Original Class: F
Encoded L

In [44]:
y_train_categorical = to_categorical(encoded_y)
y_test_categorical = to_categorical(encoded_y_test)
y_train_categorical[:10]

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [45]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(100, activation='relu'))

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               34700     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
Total params: 44,800
Trainable params: 44,800
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.add(Dense(y_train_categorical.shape[1], activation="softmax"))

In [48]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

In [49]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
743/743 - 5s - loss: 0.2791 - accuracy: 0.9439
Epoch 2/100
743/743 - 2s - loss: 0.1837 - accuracy: 0.9536
Epoch 3/100
743/743 - 2s - loss: 0.1576 - accuracy: 0.9579
Epoch 4/100
743/743 - 2s - loss: 0.1473 - accuracy: 0.9606
Epoch 5/100
743/743 - 2s - loss: 0.1367 - accuracy: 0.9640
Epoch 6/100
743/743 - 2s - loss: 0.1279 - accuracy: 0.9653
Epoch 7/100
743/743 - 2s - loss: 0.1192 - accuracy: 0.9681
Epoch 8/100
743/743 - 2s - loss: 0.1087 - accuracy: 0.9710
Epoch 9/100
743/743 - 2s - loss: 0.1068 - accuracy: 0.9724
Epoch 10/100
743/743 - 3s - loss: 0.0986 - accuracy: 0.9740
Epoch 11/100
743/743 - 2s - loss: 0.0934 - accuracy: 0.9757
Epoch 12/100
743/743 - 2s - loss: 0.0876 - accuracy: 0.9768
Epoch 13/100
743/743 - 2s - loss: 0.0835 - accuracy: 0.9798
Epoch 14/100
743/743 - 2s - loss: 0.0808 - accuracy: 0.9797
Epoch 15/100
743/743 - 2s - loss: 0.0759 - accuracy: 0.9811
Epoch 16/100
743/743 - 2s - loss: 0.0768 - accuracy: 0.9818
Epoch 17/100
743/743 - 3s - loss: 0.0680 - accura

In [50]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

248/248 - 2s - loss: 3.4151 - accuracy: 0.9351
Loss: 3.415125846862793, Accuracy: 0.9351174235343933
